In [1]:
import numpy as np
import pandas as pd

In [3]:
data = pd.read_csv("data_test.csv")

In [8]:
data = data[["Review"]]
data

,Review
0,Một hôm thèm quá tiện đường rẽ vào đây vào th...
1,mình rất hài lòng mỗi khi ăn ở không biết là ...
2,I ordered online talking to the representativ...
3,Quán khá nhỏ ko có chỗ để ngồi chỉ Ship là ch...
4,Mọi thứ hãy đến và cảm nhận
...,...
1769,Hôm mùng tết có đi qa đây mua cốc trà Topping...
1770,Trà sữa socola ngon nhất trong những quán đã ...
1771,Quán bé do quán vắng mình chó đến ngồi ngoài ...
1772,Hôm nay lần đầu đến quán cảm thấy ăn cũng nhâ...


In [11]:
!pip3 install pyvi
from pyvi import ViPosTagger, ViTokenizer

     |████████████████████████████████| 8.5MB 2.6MB/s 
     |████████████████████████████████| 747kB 37.0MB/s 


In [12]:
englishwords = pd.read_json("./englishwords.json")
englishwords.set_index("english", inplace = True)
shortform = pd.read_json("./shortform.json")
shortform.set_index("short", inplace = True)

In [13]:
import re 

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ' '))
    
    #Loại bỏ các chữ cái lặp nhau
    length = len(text)
    char = 0
    while char <length-1:
        if text[char] == text[char+1]:
            text = text[:char]+text[char+1:]
            #print(text)
            length-=1
            continue
        char+=1
    #loại bỏ số  và các chữ cái lạ:
    digits_and_characters = 'aăâbcdđeêfghijklmnoôơpqrstuưvxywzáàảãạắằẳẵặấầẩẫậéèẻẽẹếềểễệíìỉĩịóòỏõọốồổỗộớờởỡợúùủũụứừửữựýỳỷỹỵ(?::|;|=)(?:-)?(?:\)|\(|D|P)_ '
    text = ''.join([i for i in text if i in digits_and_characters])

    text_split = text.split()
    #print(text_split)
    for i in range(len(text_split)):
      t = text_split[i]
      if t in englishwords.index:
        text_split[i] = str(englishwords.loc[t, "vietnamese"])
      elif t in shortform.index:
        text_split[i] = str(shortform.loc[t,"long"]) 
    #print(text_split)
    text = ' '.join(text_split)
    return text

In [14]:
def clean(data):
  for i in data.index:
    data.loc[i, "Review"] = ViTokenizer.tokenize(preprocessor(data.loc[i, "Review"]))
  return data

In [15]:
def splitSentence(text):
  list_of_sentences = text.split('.')
  return list_of_sentences
print(splitSentence("dùng khá ổn. Cực kì ủng hộ.Ilove you"))
a,b, c = splitSentence("dùng khá ổn. Cực kì ủng hộ.Ilove you")
print(a)
print(b)
print(c)

['dùng khá ổn', ' Cực kì ủng hộ', 'Ilove you']
dùng khá ổn
 Cực kì ủng hộ
Ilove you


In [16]:
with open('./stopwords_vn_dash.txt') as f:
    stopwords_vn = f.read().splitlines()

In [17]:
data = clean(data)

In [18]:
def splitTest(X_test):
  X_test_split = pd.DataFrame(columns = ["Sentence","Origin"])
  for i in X_test.index:
    los = splitSentence(X_test.loc[i,"Review"])
    for j in range (len(los)):
      new_row = {"Sentence": los[j], "Origin": i}
      X_test_split = X_test_split.append(new_row, ignore_index= True)
  return X_test_split  

In [19]:
def concatPred(y_pred, X_test_split):
  #print(len(y_pred))
  X_test_split["Predict"] = y_pred
  X_test_split
  X_test_split = X_test_split.groupby("Origin").sum()
  y_pred = X_test_split["Predict"].tolist()
  for i in range(len(y_pred)):
    if y_pred[i] > 0:
      y_pred[i] = 1.
    elif y_pred[i] <0: 
      y_pred[i] = -1.
    else:
      y_pred[i] = 0.
  #print(len(y_pred))
  return y_pred

In [20]:
test_split = splitTest(data)
print(test_split)

                                               Sentence Origin
0     một hôm thèm quá tiện đường rẽ vào đây vào thì...      0
1     mình rất hài_lòng mỗi khi ăn ở không biết là l...      1
2     i ordered online talking to the representative...      2
3     quán khá nhỏ không có chỗ để ngồi chỉ ship là ...      3
4                           mọi thứ hãy đến và cảm_nhận      4
...                                                 ...    ...
1769  hôm mùng tết có đi qa đây mua cốc trà toping t...   1769
1770  trà sữa socola ngon nhất trong những quán đã t...   1770
1771  quán bé do quán vắng mình chó đến ngồi ngoài c...   1771
1772  hôm_nay lần đầu đến quán cảm_thấy ăn cũng nhân...   1772
1773  chị chủ và anh chủ siêu siêu dễ menu các món đ...   1773

[1774 rows x 2 columns]


In [23]:
import pickle
tfidf = pickle.load(open("tfidf_transform.sav", 'rb'))
print(tfidf)
lor = pickle.load(open("lor_model.sav", 'rb'))
print(lor)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=2, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words=['a_lô', 'a_ha', 'ai', 'ai_ai', 'ai_nấy', 'ai_đó',
                            'alô', 'amen', 'anh', 'anh_ấy', 'ba', 'ba_ba',
                            'ba_bản', 'ba_cùng', 'ba_họ', 'ba_ngày', 'ba_ngôi',
                            'ba_tăng', 'bao_giờ', 'bao_lâu', 'bao_nhiêu',
                            'bao_nả', 'bay_biến', 'biết', 'biết_bao',
                            'biết_bao_nhiêu', 'biết_chắc', 'biết_chừng_nào',
                            'biết_mình', 'biết_mấy', ...],
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
                vocabulary

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
test_split_tfidf = tfidf.transform(test_split["Sentence"])
y_pred = lor.predict(test_split_tfidf.toarray())
y_pred = concatPred(y_pred, test_split)

In [25]:
data["Label"] = y_pred
data

,Review,Label
0,một hôm thèm quá tiện đường rẽ vào đây vào thì...,-1.0
1,mình rất hài_lòng mỗi khi ăn ở không biết là l...,1.0
2,i ordered online talking to the representative...,-1.0
3,quán khá nhỏ không có chỗ để ngồi chỉ ship là ...,1.0
4,mọi thứ hãy đến và cảm_nhận,-1.0
...,...,...
1769,hôm mùng tết có đi qa đây mua cốc trà toping t...,-1.0
1770,trà sữa socola ngon nhất trong những quán đã t...,1.0
1771,quán bé do quán vắng mình chó đến ngồi ngoài c...,-1.0
1772,hôm_nay lần đầu đến quán cảm_thấy ăn cũng nhân...,1.0


In [26]:
data.to_csv("./data_test_labeled.csv")